In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./dataset/training_set_VU_DM.csv')
df_test = pd.read_csv('./dataset/test_set_VU_DM.csv')

# Dropping columns 
For starters, we can drop columns that do not provide valuable information or are missing a lot of values

- Dropping the different company attribute since the majority of the values are missing, and they seem to provide little information (the only thing that comes to mind perhaps is if someone visits expedia, therefore, they trust the brand more, and thus seeing them having more expensive options would not change their mind... WAY too hard to capture... perhaps better to drop for now)
- date_time: since no one cares about when search took place (might matter in some cases like trends and seasonality but will be also extremely hard to do)
- gross_booking: since  our model should not care about how much they spent on the hotel, and only if they purchase or not
- click_bool, booking_book: transformed into a relevant target_bool column
- srch_affinity_score: I do not understand this attribute or how it's supposed to be relevant. Will remove now, revisit late 
- srch_booking_window: Irrelevant for ranking
- prop_location_score2: missing 22%, seems very valuable, find suitable imputation method
- 'orig_destination_distance', 'srch_query_affinity_score': LOTS of missing values, Might impute later to test
- random_bool: an interesting attribute but fail to see its relevance, will revisit!

In [2]:
drop_columns_train = ['date_time', 'site_id', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff', 'click_bool', 'booking_bool', 'gross_bookings_usd', 'orig_destination_distance', 'random_bool']

drop_columns_test = ['date_time', 'site_id', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff', 'orig_destination_distance', 'random_bool']

df.drop(columns=drop_columns_train, inplace=True)
df_test.drop(columns=drop_columns_test, inplace=True)

# Experimenting with imputation
User history rating, THE majority is null. Use mean imputation to fill these values. The idea is that the majority of the people will not rate too high or too low. 
Another approache besides mean imputation is to learn the distribution of the data (for example normal distribution for the average rating) and impute based on that, so it would retain it's normal distribution.... 

In [3]:
# for column in ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_review_score']:
#     mean_value = df[column].mean()
#     df[column].fillna(mean_value, inplace=True)
# 
# for column in ['visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_review_score']:
#     mean_value = df_test[column].mean()
#     df_test[column].fillna(mean_value, inplace=True)

# Normalization

# Removing the unique identifiers

In [ ]:
unique_ids = ['visitor_location_country_id', 'srch_destination_id']

df.drop(columns=unique_ids, inplace=True)
df_test.drop(columns=unique_ids, inplace=True)

df.to_csv("./dataset/train_clean_v1_noIDs.csv", index=False)
df_test.to_csv("./dataset/test_clean_v1_noIDs.csv", index=False)

# Aggregated features 

In [4]:
def aggregated_features_single_column(
    in_data,
    key_for_grouped_by="prop_id",
    target_column="price_usd",
    agg_methods=["mean", "median", "min", "max"],
    transform_methods={"mean": ["substract"]},
):
    df = in_data.groupby(key_for_grouped_by).agg({target_column: agg_methods})

    if isinstance(key_for_grouped_by, list):
        str_key_for_grouped_by = "|".join(key_for_grouped_by)
    else:
        str_key_for_grouped_by = key_for_grouped_by

    df.columns = df.columns.droplevel()
    col = {}
    for method in agg_methods:
        col[method] = (
            method.upper() + "(" + str_key_for_grouped_by + ", " + target_column + ")"
        )

    df.rename(columns=col, inplace=True)

    in_data = in_data.merge(df.reset_index(), on=key_for_grouped_by)
    for method_name in transform_methods:
        for applying_function in transform_methods[method_name]:
            function_data = in_data[
                method_name.upper()
                + "("
                + str_key_for_grouped_by
                + ", "
                + target_column
                + ")"
            ]
            column_data = in_data[target_column]
            if applying_function == "substract":
                result = column_data - function_data
            elif applying_function == "divide":
                result = column_data / function_data
            else:
                continue
            in_data[
                applying_function.upper()
                + "("
                + target_column
                + ", "
                + method_name.upper()
                + ")"
            ] = result
    return in_data

data_for_training = aggregated_features_single_column(
        df, "prop_id", "price_usd", ["mean"]
    )
data_for_testing = aggregated_features_single_column(
        df_test, "prop_id", "price_usd", ["mean"]
    )
data_for_testing

,srch_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,"MEAN(prop_id, price_usd)","SUBSTRACT(price_usd, MEAN)"
0,1,216,NaN,NaN,219,3180,3,4.5,1,2.94,...,19222,1,10,2,0,1,0,NaN,125.520757,-6.520757
1,1,216,NaN,NaN,219,5543,3,4.5,1,2.64,...,19222,1,10,2,0,1,0,NaN,117.532579,0.467421
2,1,216,NaN,NaN,219,14142,2,3.5,1,2.71,...,19222,1,10,2,0,1,0,NaN,51.886600,-2.886600
3,1,216,NaN,NaN,219,22393,3,4.5,1,2.40,...,19222,1,10,2,0,1,0,NaN,133.987619,9.012381
4,1,216,NaN,NaN,219,24194,3,4.5,1,2.94,...,19222,1,10,2,0,1,0,NaN,86.308224,-7.308224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,216,NaN,NaN,117,32019,4,3.5,0,2.48,...,19246,2,7,1,0,1,0,NaN,62.260769,3.809231
4959179,332787,216,NaN,NaN,117,33959,4,3.0,1,2.20,...,19246,2,7,1,0,1,0,NaN,63.847692,3.252308
4959180,332787,216,NaN,NaN,117,35240,4,0.0,0,1.79,...,19246,2,7,1,0,1,0,NaN,62.955000,10.955000
4959181,332787,216,NaN,NaN,117,94437,4,0.0,0,2.94,...,19246,2,7,1,0,1,0,NaN,64.810000,1.260000


# Data output

In [ ]:
data_for_training.to_csv("./dataset/train_new_feature.csv", index=False)
data_for_testing.to_csv("./dataset/test_new_feature.csv", index=False)